<a href="https://colab.research.google.com/github/QaziSaim/Transformer-Text-Generation/blob/main/Biggest_Transformer_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install kagglehub

In [2]:
from google.colab import userdata
import os
os.environ['KAGGLE_API_KEY'] = userdata.get('kaggle_api_key')
os.environ['KAGGLE_USERNAME'] = userdata.get('kaggle_username')

### Downloading The Dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhruvildave/en-fr-translation-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'en-fr-translation-dataset' dataset.
Path to dataset files: /kaggle/input/en-fr-translation-dataset


In [5]:
# %ls /root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2
%ls /kaggle/input/en-fr-translation-dataset/en-fr.csv

/kaggle/input/en-fr-translation-dataset/en-fr.csv


In [6]:
import pandas as pd
path = '/kaggle/input/en-fr-translation-dataset/en-fr.csv'
# df = pd.read_csv('/root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2/en-fr.csv')
# df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')

In [8]:
en_file = open('english.txt','w',encoding='utf-8')
fr_file = open('french.txt','w',encoding='utf-8')

for chunk in pd.read_csv(path, chunksize=10000):
  for en, fr in zip(chunk['en'],chunk['fr']):
    if isinstance(en,str) and isinstance(fr,str):
      en_file.write(en.strip() + "\n")
      fr_file.write(fr.strip() + "\n")
en_file.close()
fr_file.close()

In [9]:
%%capture
%pip install sentencepiece


In [ ]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train(
    input='english.txt,french.txt',
    model_prefix = 'enfr',
    vocab_size=16000,
    character_coverage=1.0,
    model_type='bpe'
)